# Classification using Keras

The target values must be in a column of the dataframe, and must be a numerical datatype, so we must convert them before training.

IMPORTANT: Notice that we are importing `ImageDataGenerator` from `keras_preprocessing` instead of `keras.preprocessing`, because the people at Keras are a bunch of old reactionary dinosaurs.

See the discussion [here](https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1), and [here](https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1).

In [1]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

/home/mohit/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load data and preparation

The csv files `db.csv` contains all the metadata we have extracted from wikiart: every entry corresponds to an artwork, and the `_id` column contains the filename of the image in the `data/images/` directory associated to that artwork.

In [2]:
nrows = 5000
df = pd.read_csv("../data/db.csv",nrows=nrows,na_values="?")

Train/test split the database:

In [3]:
df_train, df_test = train_test_split(df, 
                                     test_size=0.1,
                                     shuffle=True)

In [4]:
df_train.head()

,_id,artistname,genre,image,image_size_data,style,title,year
1382,5772719aedc2cb3880c237af,camille pissarro,portrait,https://uploads0.wikiart.org/images/camille-pi...,"[{'sizekb': 8, 'width': 210, 'height': 148, 'u...",impressionism,little goose girl,1886
1316,5772719dedc2cb3880c23c05,camille pissarro,genre painting,https://uploads8.wikiart.org/images/camille-pi...,"[{'sizekb': 16, 'width': 210, 'height': 266, '...",impressionism,peasant trimming the lawn,1882
2941,5bdc8cddedc2c921c89903ad,jose guadalupe posada,caricature,https://uploads0.wikiart.org/00208/images/jose...,"[{'sizekb': 13, 'width': 210, 'height': 153, '...",modernismo,untitled,NaN
2027,577278f3edc2cb3880d96f9a,vasily polenov,landscape,https://uploads4.wikiart.org/images/vasily-pol...,"[{'sizekb': 5, 'width': 210, 'height': 111, 'u...",realism,the river oyat,1880
1473,57727196edc2cb3880c230ff,camille pissarro,genre painting,https://uploads0.wikiart.org/images/camille-pi...,"[{'sizekb': 10, 'width': 210, 'height': 170, '...",pointillism,haymakers resting,1891


Let's initialize a dataimage generator: it is a nice interface towards many (pre)processing method in Keras, including some utilities for data augmentation.

We will use the amazing `flow_from_dataframe` function to serve the data we need.

If the files do not have an extension, run this in a shell:

    $ for f in *; do mv "$f" "$f.jpg"; done

### Classes

Decide here what feature we want to predict, and save in the `classes` set all the possible values: they are the values that appear at least once in the database.

In [5]:
feature = "style"

In [6]:
classes = set(df[feature])

The number of classes will be needed later, the NN must know what's the output dimension.

In [7]:
nclass = len(classes)
print(nclass)

80


In [8]:
classes

{'abstract art',
 'abstract art,abstract expressionism',
 'abstract expressionism',
 'abstract expressionism,action painting',
 'academicism',
 'action painting',
 'art brut',
 'art informel',
 'art informel,automatic painting',
 'art informel,expressionism',
 'art informel,minimalism',
 'art informel,surrealism',
 'art nouveau (modern)',
 'art nouveau (modern),impressionism',
 'art singulier',
 'baroque',
 'classicism',
 'color field painting',
 'conceptual art',
 'conceptual art,environmental (land) art',
 'conceptual art,minimalism',
 'conceptual art,pop art',
 'concretism',
 'constructivism',
 'contemporary realism',
 'cubism',
 'cubism,expressionism',
 'cubism,futurism',
 'cubism,surrealism',
 'dada',
 'early renaissance',
 'early renaissance,international gothic',
 'environmental (land) art',
 'expressionism',
 'expressionism,muralism',
 'expressionism,pop art',
 'expressionism,social realism',
 'futurism',
 'gongbi',
 'hard edge painting',
 'hard edge painting,neoplasticism',
 '

### Image Size

The images will be scaled down to this size

In [9]:
img_size = (32,32)

## Create data generators

In [10]:
datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2)

In [11]:
train_generator = datagen.flow_from_dataframe(
                                df_train,
                                directory="../data/images/",
                                x_col="_id",
                                has_ext=False,
                                target_size=img_size,
                                y_col=feature,
                                batch_size=32,
                                classes = classes)

Found 4500 images belonging to 80 classes.


# Model Architecture: CNN with no tranfer learning

We'll start with the most basic architecture imaginable. 

It would be nice to use transfer learning.

In [12]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(*img_size,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nclass, activation='softmax'))

Pick an optimizer and compile the model

In [13]:
model.compile(
    optimizers.rmsprop(lr=0.0001, 
                       decay=1e-6),
    loss="categorical_crossentropy",
    metrics=["accuracy"])

# Training

In [14]:
model.fit_generator(generator=train_generator,
                    epochs=2)

Epoch 1/2
141/141 [==============================] - 12s 83ms/step - loss: 3.3955 - acc: 0.1863
Epoch 2/2
141/141 [==============================] - 13s 94ms/step - loss: 3.1454 - acc: 0.2374


In [17]:
test_generator = datagen.flow_from_dataframe(
                                df_test,
                                directory="../data/images/",
                                x_col="_id",
                                has_ext=False,
                                target_size=img_size,
                                y_col=feature,
                                batch_size=32,
                                classes = classes)

Found 500 images belonging to 80 classes.


# Evaluation

This returns the loss and accuracy

In [18]:
model.evaluate_generator(generator=test_generator)

[3.1134791278839113, 0.2899999997615814]